In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unsw-nb15/UNSW_NB15_testing-set.csv
/kaggle/input/unsw-nb15/UNSW-NB15_1.csv
/kaggle/input/unsw-nb15/UNSW_NB15_training-set.csv
/kaggle/input/unsw-nb15/UNSW-NB15_LIST_EVENTS.csv
/kaggle/input/unsw-nb15/UNSW-NB15_4.csv
/kaggle/input/unsw-nb15/UNSW-NB15_3.csv
/kaggle/input/unsw-nb15/UNSW-NB15_2.csv
/kaggle/input/unsw-nb15/NUSW-NB15_features.csv


Cell 1: Load và Hợp nhất Dữ liệu

In [2]:
import numpy as np
import pandas as pd

# Load dữ liệu
try:
    train_df = pd.read_csv('/kaggle/input/unsw-nb15/UNSW_NB15_training-set.csv')
    test_df = pd.read_csv('/kaggle/input/unsw-nb15/UNSW_NB15_testing-set.csv')
    print("Tải dữ liệu thành công!")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file. Hãy kiểm tra lại phần 'Add data'.")

# Hợp nhất thành một DataFrame duy nhất và xóa cột 'id'
df = pd.concat([train_df, test_df], ignore_index=True)
df = df.drop('id', axis=1)

print(f"Bộ dữ liệu UNSW-NB15 có {df.shape[0]} dòng và {df.shape[1]} cột.")
df.head()

Tải dữ liệu thành công!
Bộ dữ liệu UNSW-NB15 có 257673 dòng và 44 cột.


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,Normal,0
1,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,Normal,0
3,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,Normal,0
4,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,Normal,0


Cell 2: Tiền xử lý Dữ liệu (Áp dụng các kỹ thuật đã học)

In [3]:
# 1. Xử lý các cột dạng chữ (Categorical)
categorical_cols = ['proto', 'service', 'state']
for col in categorical_cols:
    # Lấy ra top 5 giá trị phổ biến nhất
    top_5_values = df[col].value_counts().nlargest(5).index
    # Gom tất cả các giá trị còn lại vào nhóm 'others'
    df[col] = np.where(df[col].isin(top_5_values), df[col], 'others')

# 2. Xử lý cột 'attack_cat' (gom nhóm tương tự)
# Bỏ qua 'Normal' và gom các cuộc tấn công hiếm
attack_top_values = df['attack_cat'].value_counts().nlargest(6).index
df['attack_cat'] = np.where(df['attack_cat'].isin(attack_top_values), df['attack_cat'], 'Others')

# 3. Xóa các cột tương quan/dư thừa (dựa trên phân tích của notebook tham khảo)
cols_to_drop = ['is_ftp_login', 'sbytes', 'dbytes', 'sloss', 'dloss', 'stcpb', 'dtcpb', 'tcprtt', 'response_body_len']
df = df.drop(cols_to_drop, axis=1)

# 4. Biến đổi Logarit cho các cột bị lệch
cols_to_log = ['smean', 'dmean', 'spkts', 'dpkts', 'sinpkt', 'dinpkt', 'sload', 'dload']
for col in cols_to_log:
    df[col + '_log1p'] = np.log1p(df[col])
df = df.drop(cols_to_log, axis=1) # Xóa cột gốc sau khi đã biến đổi

print("Tiền xử lý hoàn tất!")
df['attack_cat'].value_counts() # Kiểm tra lại các nhãn

Tiền xử lý hoàn tất!


attack_cat
Normal            93000
Generic           58871
Exploits          44525
Fuzzers           24246
DoS               16353
Reconnaissance    13987
Others             6691
Name: count, dtype: int64

Cell 3: Chuẩn bị cho Huấn luyện (Tách X, y và Mã hóa)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1. Mã hóa tất cả các cột dạng chữ còn lại
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# 2. Tách X và y
# 'label' là cột nhị phân (0/1), 'attack_cat' là cột đa lớp
X = df.drop(['label', 'attack_cat'], axis=1) 
y = df['attack_cat'] # Chúng ta sẽ dự đoán loại tấn công cụ thể

# 3. Chia Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 4. Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Đã sẵn sàng để huấn luyện!")

Đã sẵn sàng để huấn luyện!


CELL 4: HUẤN LUYỆN MÔ HÌNH RANDOM FOREST VỚI THAM SỐ CƠ BẢN

In [5]:
# ==============================================================================
# CELL 4: HUẤN LUYỆN MÔ HÌNH RANDOM FOREST VỚI THAM SỐ CƠ BẢN
# ==============================================================================

### THÊM MỚI: IMPORT CÁC THƯ VIỆN CẦN THIẾT CHO VIỆC HUẤN LUYỆN ###
from sklearn.ensemble import RandomForestClassifier # <-- DÒNG QUAN TRỌNG NHẤT
from sklearn.metrics import classification_report, accuracy_score
import time
import joblib # Thêm luôn để lưu file ở cell sau
import matplotlib.pyplot as plt # Thêm để vẽ biểu đồ nếu cần
import seaborn as sns # Thêm để vẽ biểu đồ nếu cần


print("--- Bắt đầu huấn luyện Mô hình Random Forest (Tham số Cơ bản) ---\n")

# Khởi tạo mô hình với các tham số bị giới hạn để làm baseline
rf_model_baseline = RandomForestClassifier(
    n_estimators=20,      # Số lượng cây ít hơn
    max_depth=10,         # Độ sâu bị giới hạn
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Ghi lại thời gian huấn luyện
start_time = time.time()
rf_model_baseline.fit(X_train_scaled, y_train)
end_time = time.time()
training_time_baseline = end_time - start_time

# Đánh giá mô hình baseline
print("\n--- Kết quả của Mô hình Cơ bản ---")
y_pred_baseline = rf_model_baseline.predict(X_test_scaled)
print("Báo cáo Phân loại (Mô hình Cơ bản):")
print(classification_report(y_test, y_pred_baseline, target_names=le.classes_, digits=4))

accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
print(f"Thời gian huấn luyện của Mô hình Cơ bản: {training_time_baseline:.2f} giây")
print(f"=> Độ chính xác tổng thể của Mô hình Cơ bản: {accuracy_baseline * 100:.4f}%\n")

--- Bắt đầu huấn luyện Mô hình Random Forest (Tham số Cơ bản) ---



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    2.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.1s finished



--- Kết quả của Mô hình Cơ bản ---
Báo cáo Phân loại (Mô hình Cơ bản):
                precision    recall  f1-score   support

           DoS     0.6228    0.0145    0.0283      4906
      Exploits     0.5812    0.9430    0.7192     13358
       Fuzzers     0.6641    0.4423    0.5309      7274
       Generic     0.9995    0.9716    0.9854     17661
        Normal     0.8811    0.9409    0.9100     27900
        Others     0.6323    0.0703    0.1265      2007
Reconnaissance     0.8133    0.6756    0.7381      4196

      accuracy                         0.8056     77302
     macro avg     0.7420    0.5797    0.5769     77302
  weighted avg     0.8094    0.8056    0.7730     77302

Thời gian huấn luyện của Mô hình Cơ bản: 2.35 giây
=> Độ chính xác tổng thể của Mô hình Cơ bản: 80.5568%



CELL 5: HUẤN LUYỆN MÔ HÌNH RANDOM FOREST VỚI THAM SỐ CẢI TIẾN

In [6]:
# ==============================================================================
# CELL 5: HUẤN LUYỆN MÔ HÌNH RANDOM FOREST VỚI THAM SỐ CẢI TIẾN
# ==============================================================================
from sklearn.metrics import classification_report, accuracy_score
import time

print("--- Bắt đầu huấn luyện Mô hình Random Forest (Tham số Cải tiến) ---\n")

# Đây là các tham số được "lựa chọn" sau quá trình khảo sát/tinh chỉnh
# để cải thiện hiệu năng so với mô hình cơ bản.
improved_params = {
    'n_estimators': 100,      # Tăng số lượng cây
    'max_depth': 50,          # Tăng độ sâu
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': 1
}

# Khởi tạo mô hình với các siêu tham số cải tiến
# Toán tử ** unpacks the dictionary into keyword arguments
rf_model_improved = RandomForestClassifier(**improved_params)

# Ghi lại thời gian huấn luyện
start_time = time.time()
rf_model_improved.fit(X_train_scaled, y_train)
end_time = time.time()
training_time_improved = end_time - start_time

# Đánh giá mô hình cải tiến
print("\n--- Kết quả của Mô hình Cải tiến ---")
y_pred_improved = rf_model_improved.predict(X_test_scaled)
print("Báo cáo Phân loại (Mô hình Cải tiến):")
print(classification_report(y_test, y_pred_improved, target_names=le.classes_, digits=4))

accuracy_improved = accuracy_score(y_test, y_pred_improved)
print(f"Thời gian huấn luyện của Mô hình Cải tiến: {training_time_improved:.2f} giây")
print(f"=> Độ chính xác tổng thể của Mô hình Cải tiến: {accuracy_improved * 100:.4f}%")


# Lưu lại mô hình tốt nhất (mô hình cải tiến)
print("\n--- Lưu lại các file .pkl từ Mô hình Cải tiến ---")
joblib.dump(rf_model_improved, 'botnet_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(le, 'label_encoder.pkl')
print("Đã lưu thành công 3 file.")

--- Bắt đầu huấn luyện Mô hình Random Forest (Tham số Cải tiến) ---



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   16.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.



--- Kết quả của Mô hình Cải tiến ---


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.8s finished


Báo cáo Phân loại (Mô hình Cải tiến):
                precision    recall  f1-score   support

           DoS     0.3424    0.2293    0.2747      4906
      Exploits     0.6378    0.8258    0.7197     13358
       Fuzzers     0.7122    0.6002    0.6514      7274
       Generic     0.9972    0.9760    0.9865     17661
        Normal     0.9142    0.9469    0.9303     27900
        Others     0.4895    0.2197    0.3033      2007
Reconnaissance     0.9139    0.7641    0.8323      4196

      accuracy                         0.8256     77302
     macro avg     0.7153    0.6517    0.6712     77302
  weighted avg     0.8191    0.8256    0.8173     77302

Thời gian huấn luyện của Mô hình Cải tiến: 16.52 giây
=> Độ chính xác tổng thể của Mô hình Cải tiến: 82.5645%

--- Lưu lại các file .pkl từ Mô hình Cải tiến ---
Đã lưu thành công 3 file.
